# Modelo Hull-White

## 1. Definiciones y Fórmulas

En la medida ajustada por riesgo la ecuación diferencial estocástica del modelo de Hull-White es la siguiente:

$$dr_{t} = (\theta_{t}-\gamma r_{t})dt+\sigma dW_{t}\quad\text(1)$$

Aquí, $\theta_{t}$ es una función de $t$ que se calibra de forma que el modelo sea capaz de replicar de forma perfecta la curva cupón cero de la fecha de proceso.
Con este modelo podemos resolver explícitamente la ecuación para el valor en $t$ del bono cupón cero que vence en $T$:

$$Z(t,T)=E\big(\exp(-\int_t^Tr_{s} \mathrm{d}s)\big)$$

Más precisamente tenemos que:

$$Z(r_{t},t,T)=A(t,T)\exp(-r_{t}B(t,T)),\forall t \in \{t>0\}\quad\text(2)$$

Donde,

$$A(t,T)=\frac{Z(0,T)}{Z(0,t)}\exp\left\{B(t,T)f(0,t)-\frac{\sigma^2}{4a}(1-\mathrm{e}^{-2\gamma t})B(t,T)^2\right\}\quad\text (3)$$

$$B(t,T)=\frac{1}{\gamma}\big(1-\mathrm{e}^{-\gamma t}\big)\quad\text(4)$$


Aquí, $Z(0,s)$ corresponde al valor del bono cupón cero (o factor de descuento) que vence en $s$ a la fecha de proceso $(t=0)$ y $f(0,t)$ corresponde a la tasa forward instantánea en $t$ a la fecha de proceso. Esta última se define como:

$$f(0,t)=-\frac{\partial \mathrm{ln}Z(0,t)}{\partial t},$$

y se puede expresar de forma más explícita como:

$$f(0,t)=r(0,t)+t\frac{\mathrm{d}r(0,t)}{\mathrm{d}t} \quad \text (5)$$

En esta última ecuación $r(0,t)$ corresponde a la tasa cupón cero que vence en $t$, en convención compuesto contínuo, a la fecha de proceso.

Finalmente, la expresión para la función $\theta_{t}$ es la siguiente:

$$\theta_{t}=\frac{\partial f(0,t)}{\partial t}+\gamma f(0,t)+\frac{\sigma^2}{2\gamma}\big(1-\mathrm{e}^{-2\gamma t}\big)\quad\text(6)$$

## 2. Calibración

Comencemos con la calibración de los parámetros $\gamma$ y $\sigma$. Realizaremos primeramente, una calibración utilizando datos históricos de la tasa corta correspondiente a la curva que se quiere modelar. Para una calibración ajustada por riesgo, se requiere que, para la curva en cuestión, existan cotizaciones de caps y floors.
Habiendo obtenido esos parámetros, se procederá a calibrar la función $\theta_{t}$.

### 2.1 Calibración Histórica de $\gamma$ y $\sigma$

Comenzamos importando la serie de datos contenida en la planilla Excel *interbancaria_clp.xlsx*. Para facilitar el trabajo con los datos, se utilizá el módulo **pandas** de Python.

In [1]:
import pandas as pd                                # Se importa el módulo pandas
data_file = pd.ExcelFile("interbancaria_clp.xlsx") # Se abre la planilla Excel con la data
data = data_file.parse("datos")                    # Se lee la data contenida en la hoja "datos"
print(type(data))                                  # La data queda almacenada en un objeto de tipo DataFrame de pandas
print()                                            # Se deja una línea en blanco
data.head()                                        # Se muestra los primeros cinco datos

<class 'pandas.core.frame.DataFrame'>
()


,Fecha,Promedio
0,2001-08-09,0.0650
1,2001-08-10,0.0650
2,2001-08-13,0.0650
3,2001-08-14,0.0650
4,2001-08-16,0.0642


In [2]:
data.tail()    # Se muestra los últimos cinco datos

,Fecha,Promedio
3991,2017-08-07,0.025
3992,2017-08-08,0.025
3993,2017-08-09,0.025
3994,2017-08-10,0.025
3995,2017-08-11,0.025


Teniendo ya la data disponible, calculamos la desviación estándar de sus retornos. Es importante notar que, dado que el modelo de Hull-White utiliza un ruido browniano aritmético, los retornos $R_{t}$ deben calcularse como la diferencia entre los valores de tasa, es decir:

$$R_{t}=r_{t}-r_{t-\Delta t}$$

Para hacer este cálculo, **pandas** dispone del método `diff`. Lo aplicaremos y obtendremos un nuevo **DataFrame** con los retornos buscados.

In [3]:
returns = data.diff()    # En returns se almacenan las diferencias diarias
returns.head()           # Se muestra los primeros 5 datos de returns

,Fecha,Promedio
0,NaT,NaN
1,1 days,0.0000
2,3 days,0.0000
3,1 days,0.0000
4,2 days,-0.0008


Finalmente, se calcula la desviación estándar sobre la serie de retornos aritméticos.

In [4]:
import math                                 # Se requiere el módulo math para utilizar sqrt
sigma_object = returns.std(ddof=1)          # Sobre el objeto returns se calcula la desviación estándar
print(type(sigma_object))                   # Se imprime el tipo de objeto que resulta del cálculo anterior
sigma = sigma_object[1] * math.sqrt(264)    # Se almacena aparte el valor de sigma anualizado
print("sigma: " + str(sigma))               # Se imprime el valor de la desviación estándar

<class 'pandas.core.series.Series'>
sigma: 0.0153226935088


Para el cálculo de $\gamma$, se recurrirá a una regresión lineal asumiendo que la función $\theta_{t}$ es constante. Esto es equivalente a suponer que el modelo de Hull-White se reduce al modelo de Vasicek $(dr_{t}=\gamma (\bar{r}-r_{t})dt+\sigma dW_{t})$.

Para plantear la regresión se discretiza la ecuación (1) suponiendo $\theta_{t}$ constante, es decir consideramos la ecuación:

$$dr_{t}=\big(\overline{r}-\gamma r_{t}\big)dt+\sigma dW_{t}$$

La cual, discretizada con el esquema de Euler, se reduce a:

$$r_{t+\Delta t}= \overline{r}\Delta t +\big(1- \gamma\Delta t\big) r_{t} + \sigma \sqrt{\Delta t}\epsilon_{t}$$

Por lo tanto, al regresionar $r_{t+\Delta t}$ versus $r_{t}$ el coeficiente $m$ que representa la pendiente de la regresión es tal que:

$$m=1-\gamma\Delta t$$

Y, por lo tanto,

$$\gamma = \frac{1-m}{\Delta t}$$

In [5]:
from scipy import stats
def vasicek_regression(rates, dt):
    x = rates[:-1]
    y = rates[1:]
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    gamma = (1 - slope) / dt
    return gamma

In [6]:
dt = 1 / 264.0                                      # 1 día suponiendo que 1 año tiene 264 días hábiles
rates = data['Promedio'].tolist()                   # Se extrae la columna con la data numérica en formato list    
gamma = vasicek_regression(rates, dt)
print("gamma:", gamma)

('gamma:', 0.45803265685643257)


### 2.2 Calibración de la Función $\theta_{t}$

La calibración de la función $\theta_{t}$ se divide en los siguientes pasos:

- Obtención de la data de la curva cupón cero
- Representación de la curva a través de una función continua y diferenciable
- Cálculo de $f(0,t)$ y $\frac{\mathrm{d}f(0,t)}{\mathrm{d}t}$
- Determinación final de la función $\theta_{t}$

#### 2.2.1 Obtención de la Data

En este caso, la data de la curva está almacenada en un archivo csv. Cargaremos este archivo en un nuevo **DataFrame** de **pandas**.

In [47]:
# Se carga el csv en el DataFrame curva
curva = pd.read_csv("./curva_2.csv")

# Se muestra toda la data. Los plazos están en años y la curva está expresada
# en términos de factores de descuento (df).
curva

,Plazo,Df
0,0.002740,0.999917
1,0.005479,0.999833
2,0.254795,0.992906
3,0.506849,0.986198
4,0.758904,0.979555
5,1.005479,0.973384
6,1.506849,0.959941
7,2.005479,0.944978
8,3.010959,0.912039
9,4.008219,0.877304


Agregamos una columna a **curva** con la tasa en composición compuesta contínua. Es decir $df=\mathrm{e}^{-rt}$, por lo tanto, $r=-\frac{\mathrm{log}(Df)}{t}$. Para hacerlo rápidamente vamos a importar **numpy**, librería que nos permite entre otras cosas, aplicar funciones a arreglos (como la columna Df de **curva**).

In [48]:
# Se importa numpy con el nombre breve np  
import numpy as np

# Se calcula la curva tasa
curva['Tasa'] = -np.log(curva['Df'])/curva['Plazo']

# Se imprime el DataFrame curva con la nueva columna
curva                                                  

,Plazo,Df,Tasa
0,0.002740,0.999917,0.030415
1,0.005479,0.999833,0.030415
2,0.254795,0.992906,0.027941
3,0.506849,0.986198,0.027422
4,0.758904,0.979555,0.027219
5,1.005479,0.973384,0.026829
6,1.506849,0.959941,0.027131
7,2.005479,0.944978,0.028220
8,3.010959,0.912039,0.030579
9,4.008219,0.877304,0.032658


Aprovechamos de hacer un gráfico de la curva, para eso utilizamos la librería **bokeh**. Definimos una función que nos permite hacer un gráfico de línea que usaremos varias veces en este cuaderno.

In [50]:
# De la librería bokeh se importa las componentes necesarias
from bokeh.plotting import figure, show, output_notebook

def plot_line(x, y, titulo, label_x, label_y):
    """
    x: list con las variables independientes
    y: list con las variables dependientes
    titulo: título del gráfico
    label_x: etiqueta del eje x
    label_y: etiqueta del eje y
    """
    # output hacia el notebook
    output_notebook()

    # create a new plot with a title and axis labels
    p = figure(title=titulo, x_axis_label=label_x, y_axis_label=label_y)

    # add a line renderer with legend and line thickness
    p.line(x, y, line_width=2)

    # show the results
    show(p)
    
# Se define la list con los plazos
plazos = curva['Plazo'].tolist()

#  Se define la list con las tasas
tasas = curva['Tasa'].tolist()

# Se grafica la curva
plot_line(plazos, tasas, 'Curva Cero', 'Plazo', 'Tasa')

Loading BokehJS ...

Recordemos la expresión para Theta:

$$\theta_{t}=\frac{\partial f(0,t)}{\partial t}+\gamma f(0,t)+\frac{\sigma^2}{2\gamma}\big(1-\mathrm{e}^{-2\gamma t}\big)\quad\text(6)$$

Necesitamos ahora una representación en tiempo contínuo de la curva para poder calcular $f(0,t)$ y su derivada. Para eso, vamos a utilizar un **cubic-spline**. Importaremos las funciones del archivo ```clamped_spline.py```.

In [10]:
# Se importa el archivo
import clamped_spline as csp

# Las listas plazos y tasas se transforman en np.arrays
plazos = np.array(plazos)
tasas = np.array(tasas)

# Se define la curva como un spline cúbico, notar que volvemos a utilizar
# las list plazos y tasas definidas más arriba, pero esta vez, se transforman
# a numpy.array.
zrate = csp.get_clamped_spline_interpol(plazos, tasas)    
                                                                      

Podemos ver como ahora podemos obtener un valor de tasa para valores arbitrarios de $t$. La función `zrate` devuelve el valor interpolado así como la derivada primera y la derivada segunda respecto a $t$.

In [13]:
# Definimos varios plazos y calculamos el valor de la tasa y sus derivadas a ese plazo
t = 1.1
print(zrate(t))

t = 5.7
print(zrate(t))

t = 18.988
print(zrate(t))

(0.026727501772866702, -0.0010979442958310016, 0.00033996688915491433)
(0.03631328426974955, 0.0036752879549880896, 0.0026150420062465325)
(0.046389714231510802, 0.0018924905997153715, 0.00041220585610963697)


Si se recuerda las ecuaciones (5) y (6), vemos que es necesario calcular $f(0,t)$ y $\frac{\mathrm{d}f(0,t}{\mathrm{d}t}$. Estas dos cantidades dependen de las derivadas primera y segunda de la función ```zrate(t)```que ésta misma devuelve.

Con lo anterior podemos definir las funciones $f(0,t)$ y $\frac{\mathrm{d}f(0,t)}{\mathrm{d}t}$. En particular, a partir de la fórmula (5) podemos deducir que:

$$\frac{\mathrm{d}f(0,t)}{\mathrm{d}t}=2\frac{\mathrm{d}r(0,t)}{\mathrm{d}t}+t\frac{\mathrm{d}^2r(0,t)}{\mathrm{d}t}$$

Tenemos entonces que:

In [14]:
def fwd(t):
    """
    Tasa forward instantánea en t
    t: tiempo en años
    """
    return zrate(t)[0] + t * zrate(t)[1]

def dfwd(t):
    """
    Derivada de la tasa forward instantánea
    t: tiempo en años
    """
    return 2 * zrate(t)[1] + t * zrate(t)[2]

print(zrate(t))    # Valor de la tasa en t (mismo t definido más arriba)
print(fwd(t))      # Valor de la tasa forward instantánea en t
print(dfwd(t))     # Derivada de la tasa forward instatánea

(0.046389714231510802, 0.0018924905997153715, 0.00041220585610963697)
0.0823243257389
0.0116119459952


In [15]:
# Con esto, podemos ya escribir una expresión para la función Theta.
# Definamos un valor para sigma y gamma
sigma = .015
gamma = .5
r0 = tasas[0]

def theta(t):
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux
print(theta(t))

0.0529991088634


In [16]:
tiempos = []
thetas = []
dt = 1 / 264.0
for i in range(0, 2641):
    t = i * dt
    tiempos.append(t)
    thetas.append(theta(t))
    
plot_line(tiempos, thetas, "Función Theta", "Tiempo", "Theta")

Loading BokehJS ...

### 3. Simulación

Habiendo construido la función $\theta_{t}$ se puede ya simular. Para esto se escribe la función `sim_hw` que retorna $numpaths$ caminos de simulación con $numsteps$ pasos de amplitud $dt$.

In [27]:
# Desde numpy importamos el módulo rnd que permite
# la generación de números aleatorios.
from numpy import random as rnd
def sim_hw(gamma, sigma, theta, r0, num_paths, num_steps, seed):
    rnd.seed(seed)
    dt = 1 / 264.0
    sqdt = math.sqrt(dt)
    sim = np.zeros([num_paths, num_steps + 1])
    for j in range(0, num_paths):
        sim[j, 0] = r0
        r = r0
        for i in range(1, num_steps + 1):
            r = r + (theta(i * dt) - gamma * r) * dt + sigma * sqdt * rnd.normal()
            sim[j, i] = r
    return sim

Se define, además, la función `plot_simulation` que permite graficar uno cualquiera de los caminos simulados.

In [28]:
def plot_simulation(dt, sim):
    # se define el eje tiempo
    tiempo = [0]
    for i in range(1, len(sim)):
        tiempo.append(i * dt)

    # output hacia el notebook
    output_notebook()

    # create a new plot with a title and axis labels
    p = figure(title="Simulación", x_axis_label='Tiempo', y_axis_label='Tasa')

    # add a line renderer with legend and line thickness
    p.line(tiempo, sim, legend="Curva", line_width=2)

    # show the results
    show(p)

In [33]:
# Se llama la función sim_hw; antes se definen los valores
# para el números de caminos, el números de pasos de cada
# camino y el valor de la semilla.
num_paths = 10
num_steps = 264
seed = 12345678
sim = sim_hw(gamma, sigma, theta, r0, num_paths, num_steps, seed)

In [35]:
# Se utiliza esta variable para indicar que simulación
# se quiere graficar.
which_sim = 4

# Se genera y despliega el gráfico.
plot_simulation(dt, sim[which_sim])

Loading BokehJS ...

### 4. Obtención de Factores de Descuento

Una vez obtenida una simulación, las expresiones (2), (3) y (4) permiten calcular el factor de descuento en cualquier instante $t$ de la misma y para cualquier plazo $t>T$ (los plazos $t$ y $T$ se miden respecto al instante inicial de la simulación $t=0$ que, a su vez, debe coincidir con la fecha de proceso.

Se definen las funciones `b_hw`, `a_hw` y `zero_hw`.

In [36]:
# Coeficiente B del modelo de HW
def b_hw(gamma, t, T):
    """
    gamma : para 
    sigma: 
    t : plazo de....
    T: plazo ....
    """
    aux = 1 - math.exp(- gamma * (T - t))
    return aux / gamma

In [40]:
# Coeficiente A del modelo de HW
def a_hw(zrate, fwd, gamma, sigma, t, T, verbose = False):
    """
    verbose: cuando es True imprime los valores de c1, c2 y c3.
    """
    b = b_hw(gamma, t, T)
    # En esta zona quedó un error en clase. Se estaba calculando
    # el cociente de las tasas cero y debe ser el cociente entre los
    # factores de descuento a T y t.
    dfT = math.exp(-zrate(T)[0] * T)
    dft = math.exp(-zrate(t)[0] * t)
    c1 = math.log(dfT / dft)
    c2 = b * fwd(t)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t))
    if verbose:
        print("c1: " + str(c1))
        print("c2: " + str(c2))
        print("c3: " + str(c3))
    return c1 + c2 - c3

In [41]:
# Factor de descuento según HW
def zero_hw(r, gamma, sigma, zrate, fwd, t, T):
    a = a_hw(zrate, fwd, gamma, sigma, t, T)
    b = b_hw(gamma, t, T)
    return math.exp(a - b * r)

In [45]:
r0 = zrate(1/365)[0]
print("r0: " + str(r0))
t = 0
index_T = 7
T = plazos[index_T]
z = zero_hw(r0, gamma, sigma, zrate, fwd, t, T)
print("t:", t, "T:", T)
print("zero_hw: " + str(round(z, 8)))
print("zero curva: " + str(round(curva['Df'][index_T], 8)))

r0: 0.030415398534
('t:', 0, 'T:', 2.0054794520547898)
zero_hw: 0.94497783
zero curva: 0.94497783
